In [ ]:
import pandas
import numpy as np
import random
import math
from sklearn.preprocessing import StandardScaler

In [ ]:
test=pandas.read_csv("../data/df_test.csv")
train=pandas.read_csv("../data/df_train.csv")

In [ ]:
def matrix_data(tt): 
    tt_nl=tt.loc[:, tt.columns != "type"]
    tt_l=tt.loc[:, tt.columns == "type"]
    thing=[[adiff(float(j)) for j in i] for i in tt_nl.values]
    return(thing,tt_l)

In [ ]:
train.shape

(149, 10)

In [ ]:
from collections import Counter

In [ ]:
print(Counter(test["type"]))

Counter({2: 23, 1: 21, 7: 9, 3: 5, 5: 4, 6: 3})


In [ ]:
##normalise data before 

In [ ]:
def next_layer(previous_layer,relevant_weights):
    bias=0.5
    train_nl,train_l=matrix_data(train)
    weights=np.random.rand(train_nl.shape[1],10)
    activation(train_nl@weights+bias)

In [ ]:
import traceback
class adiff:
    
    def __init__(self, value, parents=None):
        try:
            assert type(value) in {float, int}, "Numbers please"
        except AssertionError:
            print(traceback.format_exc()) 
            print(type(value))           
        self.val = value
        self.gradient = 0.0
        self.parents = parents
        if parents == None:
            self.parents = []

    """The backprop, from Rasmusbergpalm's github"""
    # def backprop(self, bp):
    #     self.grad += bp
    #     for parent, grad in self.parents:
    #         parent.backprop(grad * bp)
    #     return self
    def backprop(self, value):
        self.gradient+= value
        for parent, gradient in self.parents:
            parent.backprop(gradient * value)
    
    def backward(self):
        self.backprop(1.0)

    def __add__(self, o_value):
        n_value = self.val + o_value.val
        return adiff(n_value, [(self, 1.0), (o_value, 1.0)])

    def __mul__(self, o_value):
        n_value = self.val * o_value.val
        return adiff(n_value, [(self, o_value.val), (o_value, self.val)])

    def __sub__(self, o_value):
        n_value = adiff(self.val - o_value.val, [(self, 1.0), (o_value, 1.0)])
        return n_value
        
    def __pow__(self, o_value):
        n_value = self.val**o_value.val
        return adiff(n_value, [(self, o_value.val*self.val**(o_value.val-1.0)), (o_value, n_value*math.log(self.val))])
    
    def __truediv__(self, o_value):
        n_value = self.val/o_value.val
        return adiff(n_value,[(self,1.0/o_value.val), (o_value,-self.val/o_value.val**2)])##!!!!!!

    #Returning a f-string, with the value and gradient
    def __repr__(self):
        return (f"adiff value = {self.val}, parents = {len(self.parents)}, gradient = {self.gradient}")

    def e(self):
        n_value=float(np.exp(self.val))
        return adiff(n_value, [(self, n_value)])
    
    def relu(self,data):
        return [[adiff(j.val,[(j,1.0)]) if j.val>0 else adiff(0,[(j,0.0)]) for j in i] for i in data]    

    def softmax(self,data):
        values=[[adiff(item.val) for item in row] for row in data]
        sums=[]
        for row in values:
            current_sum = adiff(0.0)
            for value in row:
                current_sum += value.e()
            sums.append(current_sum)
        return [[(data[ri][ci]).e()/sums[ri] for ci in range(len(data[0]))]for ri in range(len(data))]

    def sigmoid(self, data):
        #print("in sigmoid",data)
        thing=[]
        for i in data:
            #print("i",i)
            thing2=[]
            for j in i:
                # print("heyo")
                #print(j.e())
                #print(adiff(1.0)/(adiff(1.0)+j.e()))
                # print("this wont print")
                thing2.append(adiff(1.0)/(adiff(1.0)+j.e()))
            thing.append(thing2)
        return thing #[[adiff(1.0)/(adiff(1.0)+j.e()) for j in i] for i in data] 
## 1.0 / (1.0 + np.exp(-data))
##sigmoid 


In [ ]:
class network:
    def __init__(self, layers_info,activate = "sigmoid"):
        #self.n_features = n_features
        self.bias = 0.5
        self.layers_info = layers_info
        self.Network=[]
        self.activate=activate
    
    def forward_pass(self,input_data):
        for row in range(len(input_data)):#add a column of 1s to add the bias during matrix multiplicaiton
            input_data[row].append(adiff(1.0))
        for l_info in self.layers_info:    #create new layers 
            new_layer = layer(l_info, len(input_data[1]),self.activate)
            new_layer.calculation(input_data)
            self.Network.append(new_layer)
            input_data = new_layer.next_layer_input
        output_layer= layer(6, len(input_data[1]),"softmax")
        output_layer.calculation(input_data)
        self.Network.append(output_layer)

    def predict(self,data):
        for i in self.Network:
            i.calculation(data)
            data=i.next_layer_input
        values=[[item.val for item in row] for row in data]
        print([sum(row)for row in values])
        return np.argmax(values,axis=1)
            
    def mse_loss(self,y_true, y_pred):
        return ((y_true-y_pred)**2).mean()
        
    def get_layer(self, i): #do we need this?
        return self.network[i]
    

class layer:
    def __init__(self,number_of_neurons, n_prev_neurons,activate):
        self.n = number_of_neurons
        self.weights = [[adiff(random.random()) for i in range(number_of_neurons)] for _ in range(n_prev_neurons+1)]
        self.next_layer_input = None
        self.activate=activate
    
    def activation(self,data):
        if self.activate == "ReLU": #check if working later
            return adiff.relu(self,data)##is self needed here check later
        elif self.activate == "sigmoid":
            return  adiff.sigmoid(self,data)
        elif self.activate == "softmax":
            return adiff.softmax(self,data)
            return np.exp(data)/(np.exp(data).sum(axis=1).reshape(-1,1))
        else:
            print("invalid activaiton function name")
        #more here if needed
        #choose activation function here

    def _dot_llist(self,llist1,llist2):
        dot_product = [[0.0 for i in range(len(llist2[0]))] for j in range(len(llist1))]
        for row in range(len(llist1)):
            for column in range(len(llist2[0])):
                a = llist1[row]
                b = [l2row[column] for l2row in llist2]
                sum_ = adiff(0.0)
                for f in a:
                    for g in b:
                        sum_ += f * g
                dot_product[row][column]=sum_
        return dot_product

    def calculation(self, data):
        self.next_layer_input = self.activation(self._dot_llist(data,self.weights))

In [ ]:
norm = StandardScaler()
tt_nl=train.loc[:, train.columns != "type"]
x = norm.fit_transform(tt_nl)#do this before making adiff later
x=[[adiff(float(j)) for j in i] for i in x]
testingthing=network([2,3],"sigmoid")
testingthing.forward_pass(x)

In [ ]:
testingthing.predict(x) #predicted labels

[1.0, 1.0000000000000002, 1.0, 1.0000000000000002, 0.9999999999999999, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999999999999, 1.0, 1.0, 0.9999999999999999, 0.9999999999999999, 1.0, 1.0, 1.0, 1.0000000000000002, 1.0, 0.9999999999999998, 0.9999999999999999, 1.0, 0.9999999999999999, 1.0, 1.0, 1.0000000000000002, 0.9999999999999999, 1.0, 0.9999999999999998, 0.9999999999999998, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999999999998, 1.0, 0.9999999999999998, 1.0, 1.0, 0.9999999999999998, 1.0, 1.0, 1.0000000000000002, 1.0, 1.0000000000000002, 1.0, 1.0, 1.0, 0.9999999999999998, 0.9999999999999999, 1.0, 1.0, 1.0000000000000002, 1.0, 1.0, 1.0000000000000002, 0.9999999999999999, 1.0, 1.0, 1.0, 1.0000000000000002, 0.9999999999999998, 1.0, 1.0, 1.0, 0.9999999999999999, 1.0, 1.0, 1.0000000000000002, 1.0, 1.0, 0.9999999999999999, 1.0, 0.9999999999999998, 1.0, 1.0, 1.0, 1.0, 1.0000000000000002, 0.9999999999999999, 1.0, 1.0, 1.0, 1.0, 0.9999999999999999, 0.9999999999999999, 1.0, 1.0, 1.0, 0.9999999999999998,

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
np.ravel(matrix_data(train)[1])#real labels 

array([2, 2, 2, 2, 1, 2, 2, 5, 2, 7, 1, 2, 7, 1, 1, 1, 7, 2, 2, 1, 2, 7,
       7, 6, 3, 2, 6, 2, 6, 7, 1, 2, 6, 2, 7, 1, 2, 1, 1, 2, 7, 2, 1, 1,
       2, 1, 3, 2, 2, 2, 2, 1, 7, 7, 1, 5, 1, 1, 1, 1, 1, 3, 1, 2, 3, 7,
       2, 2, 2, 1, 3, 6, 5, 2, 1, 6, 2, 2, 7, 2, 1, 1, 3, 5, 2, 1, 1, 1,
       1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 3, 2, 2, 1, 3, 1, 3, 7, 2, 5, 1,
       2, 1, 1, 1, 7, 3, 1, 3, 5, 2, 2, 5, 7, 2, 1, 7, 2, 3, 7, 7, 2, 1,
       7, 1, 1, 2, 2, 2, 5, 2, 5, 1, 2, 1, 1, 2, 7, 1, 1])

In [ ]:
#https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/
#^ for later reference for the cross entropy loss we need to use

- [ ] check relu works
- [ ] random seed?


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=58866f15-1cff-4fef-9525-5c3070562370' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>